In [21]:
# Import dependencies
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from xgboost.sklearn import XGBClassifier  
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [22]:
feature_csv = "feature_df.csv"
target_csv = "target_df.csv"

feature_csv_df = pd.read_csv(feature_csv)
target_csv_df = pd.read_csv(target_csv)

In [23]:
feature_csv_df.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_steff_err1,koi_model_snr
0,0,0,0,81,25.8
1,0,1,0,158,76.3
2,0,1,0,157,505.6
3,0,0,0,169,40.9
4,0,0,0,189,40.2


In [24]:
feature_csv_df.shape

(6991, 5)

In [25]:
target_csv_df.head()

,koi_disposition
0,CONFIRMED
1,FALSE POSITIVE
2,FALSE POSITIVE
3,CONFIRMED
4,CONFIRMED


In [26]:
target_csv_df.shape

(6991, 1)

# Split, nornmalize, and encode the the data

In [27]:
# Assign the features and target to X and y abd check assignments
raw_feature_data = feature_csv_df.values
raw_target_data = target_csv_df.values
X = raw_feature_data[:, 0:6]
y = raw_target_data[:, 0]

print(X, y)

[[  0.    0.    0.   81.   25.8]
 [  0.    1.    0.  158.   76.3]
 [  0.    1.    0.  157.  505.6]
 ...
 [  0.    0.    0.  165.   10.6]
 [  0.    0.    1.  193.   12.3]
 [  0.    0.    1.  158.    8.2]] ['CONFIRMED' 'FALSE POSITIVE' 'FALSE POSITIVE' ... 'CANDIDATE'
 'FALSE POSITIVE' 'FALSE POSITIVE']


In [28]:
# Create the train and test sets for the features and target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [29]:
# Scale the features using the MinMax scaler since we know their values
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Encode the target for the train and test set so it can be fed into our NN and conver to a vector and check it
label_encoder = LabelEncoder()
label_encoder.fit(y_train)


encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

encoded_y_train

array([0, 2, 2, ..., 2, 2, 2])

In [31]:
# Convert encoded targets to one-hot-encoding and check it
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

y_train_categorical

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [32]:
# Import the sequential and dense modules fo build my NN
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm

# HYPERPERAMETER TUNING

In [33]:
# Create the base Keras classifier wrapper for activation method
def create_base_model(activation='relu'):
    
    model = Sequential()
    model.add(Dense(units=100, activation='relu', input_dim=5))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=100, activation='tanh'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


# Create activation model
model = KerasClassifier(build_fn=create_base_model, epochs=250, batch_size=88)


# Define the activation grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']


# Assign the activation grid search parameters, fit, and run the model
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=activation_param_grid, n_jobs=-1)
grid_result = activation_grid.fit(X_train_scaled, y_train_categorical)

C:\Users\howar\Anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/250
5243/5243 [==============================] - 0s 64us/step - loss: 0.6410 - acc: 0.6348
Epoch 2/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.4455 - acc: 0.7557
Epoch 3/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.4357 - acc: 0.7761
Epoch 4/250
5243/5243 [==============================] - 0s 15us/step - loss: 0.4245 - acc: 0.8039
Epoch 5/250
5243/5243 [==============================] - 0s 15us/step - loss: 0.4182 - acc: 0.8074
Epoch 6/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.4167 - acc: 0.8081
Epoch 7/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.4177 - acc: 0.8072
Epoch 8/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.4128 - acc: 0.8097
Epoch 9/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.4119 - acc: 0.8104
Epoch 10/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.4093 - acc: 0.8142
Epoch 11/

5243/5243 [==============================] - 0s 14us/step - loss: 0.3311 - acc: 0.8627
Epoch 84/250
5243/5243 [==============================] - 0s 15us/step - loss: 0.3338 - acc: 0.8600
Epoch 85/250
5243/5243 [==============================] - 0s 15us/step - loss: 0.3268 - acc: 0.8627
Epoch 86/250
5243/5243 [==============================] - 0s 15us/step - loss: 0.3258 - acc: 0.8594
Epoch 87/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.3281 - acc: 0.8650
Epoch 88/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3248 - acc: 0.8634
Epoch 89/250
5243/5243 [==============================] - 0s 15us/step - loss: 0.3324 - acc: 0.8644
Epoch 90/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.3363 - acc: 0.8550
Epoch 91/250
5243/5243 [==============================] - 0s 15us/step - loss: 0.3295 - acc: 0.8608
Epoch 92/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.3246 - acc: 0.8644
Epoch 93/250


5243/5243 [==============================] - 0s 13us/step - loss: 0.3240 - acc: 0.8623
Epoch 165/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3198 - acc: 0.8629
Epoch 166/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3206 - acc: 0.8665
Epoch 167/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3215 - acc: 0.8652
Epoch 168/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3195 - acc: 0.8661
Epoch 169/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3225 - acc: 0.8650
Epoch 170/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3230 - acc: 0.8650
Epoch 171/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3205 - acc: 0.8667
Epoch 172/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3278 - acc: 0.8632
Epoch 173/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3188 - acc: 0.8638
Epoc

5243/5243 [==============================] - 0s 14us/step - loss: 0.3190 - acc: 0.8665
Epoch 246/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3175 - acc: 0.8636
Epoch 247/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3139 - acc: 0.8688
Epoch 248/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3190 - acc: 0.8667
Epoch 249/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3175 - acc: 0.8697
Epoch 250/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3152 - acc: 0.8644


In [36]:
# Print the score and best params for the activation model
print("Best: %f using %s" % (activation_grid_result.best_score_, activation_grid_result.best_params_))

Best: 0.864009 using {'activation': 'linear'}


In [37]:
print(f"Training Data Score: {activation_grid_result.score(X_train_scaled, y_train_categorical)}")
print(f"Testing Data Score: {activation_grid_result.score(X_test_scaled, y_test_categorical)}")

5243/5243 [==============================] - 0s 6us/step
Training Data Score: 0.8680144972913074
1748/1748 [==============================] - 0s 6us/step
Testing Data Score: 0.8649885567156619


# SAVE THE MODEL

In [40]:
# Save the tuned model for future use
import joblib
filename = 'Howard_Mitchell_Sequential_NN.sav'
joblib.dump(grid_result, filename)

['Howard_Mitchell_Sequential_NN.sav']

# THE BELOW CONTAINS DIFFERENT GRID SEARCHS I WAS TESTING OUT
# KEEPING THIS IN THE NOTEBOOK FOR FUTURE REFERENCE

In [ ]:
# # Create the base Keras classifier wrapper for the model
# def create_model(activation='adam', dropout_rate=0.0):
    
#     model = Sequential()
#     model.add(Dense(units=100, activation='relu', input_dim=5))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=100, activation='tanh'))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=3, activation='softmax'))
#     model.add(Dropout(dropout_rate))
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
#     return model


# # Create model
# model = KerasClassifier(build_fn=create_model, epochs=250, batch_size=88)


# # Define the activation and dropout rate grid search parameters
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


# # Assign the grid search parameters, fit, and run the model
# param_grid = dict(activation=activation, dropout_rate=dropout_rate)
# fit_grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# grid_result = fit_grid.fit(X_train_scaled, y_train_categorical)

In [ ]:
# # Print the score and best params for the activation model
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [17]:
# # Create the base Keras classifier wrapper for optimizer model
# def create_optimizer_model(optimizer='adam'):
    
#     model = Sequential()
#     model.add(Dense(units=100, activation='hard_sigmoid', input_dim=5))
#     model.add(Dense(units=100, activation='hard_sigmoid'))
#     model.add(Dense(units=100, activation='hard_sigmoid'))
#     model.add(Dense(units=100, activation='hard_sigmoid'))
#     model.add(Dense(units=3, activation='hard_sigmoid'))
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
#     return model


# # Create optimizer model
# optimizer_model = KerasClassifier(build_fn=create_optimizer_model, epochs=250, batch_size=88)


# # Define the optimizer grid search parameters
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']


# # Assign the optimizer grid search parameters, fit, and run the model
# optimizer_param_grid = dict(optimizer=optimizer)
# optimizer_grid = GridSearchCV(estimator=optimizer_model, param_grid=optimizer_param_grid, n_jobs=-1)
# optimizer_grid_result = optimizer_grid.fit(X_train_scaled, y_train_categorical)

Epoch 1/250
5243/5243 [==============================] - 0s 73us/step - loss: 1.0401 - acc: 0.5056
Epoch 2/250
5243/5243 [==============================] - 0s 17us/step - loss: 1.0368 - acc: 0.5056
Epoch 3/250
5243/5243 [==============================] - 0s 16us/step - loss: 1.0372 - acc: 0.5056
Epoch 4/250
5243/5243 [==============================] - 0s 16us/step - loss: 1.0376 - acc: 0.5056
Epoch 5/250
5243/5243 [==============================] - 0s 16us/step - loss: 1.0378 - acc: 0.5056
Epoch 6/250
5243/5243 [==============================] - 0s 16us/step - loss: 1.0374 - acc: 0.5056
Epoch 7/250
5243/5243 [==============================] - 0s 16us/step - loss: 1.0373 - acc: 0.5056
Epoch 8/250
5243/5243 [==============================] - 0s 16us/step - loss: 1.0369 - acc: 0.5056
Epoch 9/250
5243/5243 [==============================] - 0s 16us/step - loss: 1.0372 - acc: 0.5056
Epoch 10/250
5243/5243 [==============================] - 0s 16us/step - loss: 1.0370 - acc: 0.5056
Epoch 11/

5243/5243 [==============================] - 0s 15us/step - loss: 7.9684 - acc: 0.5056
Epoch 84/250
5243/5243 [==============================] - 0s 15us/step - loss: 7.9684 - acc: 0.5056
Epoch 85/250
5243/5243 [==============================] - 0s 15us/step - loss: 7.9684 - acc: 0.5056
Epoch 86/250
5243/5243 [==============================] - 0s 15us/step - loss: 7.9684 - acc: 0.5056
Epoch 87/250
5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoch 88/250
5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoch 89/250
5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoch 90/250
5243/5243 [==============================] - 0s 17us/step - loss: 7.9684 - acc: 0.5056
Epoch 91/250
5243/5243 [==============================] - 0s 17us/step - loss: 7.9684 - acc: 0.5056
Epoch 92/250
5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoch 93/250


5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoch 165/250
5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoch 166/250
5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoch 167/250
5243/5243 [==============================] - 0s 15us/step - loss: 7.9684 - acc: 0.5056
Epoch 168/250
5243/5243 [==============================] - 0s 15us/step - loss: 7.9684 - acc: 0.5056
Epoch 169/250
5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoch 170/250
5243/5243 [==============================] - 0s 15us/step - loss: 7.9684 - acc: 0.5056
Epoch 171/250
5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoch 172/250
5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoch 173/250
5243/5243 [==============================] - 0s 16us/step - loss: 7.9684 - acc: 0.5056
Epoc

In [18]:
# # Print the score and best params for the optimizer model
# print("Best: %f using %s" % (optimizer_grid_result.best_score_, optimizer_grid_result.best_params_))

Best: 0.750715 using {'optimizer': 'Adadelta'}


In [21]:
# # Create the base Keras classifier wrapper for dropout model
# def create_dropout_model(dropout_rate=0.0):
    
#     model = Sequential()
#     model.add(Dense(units=100, activation='relu', input_dim=5))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=100, activation='tanh'))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=3, activation='softmax'))
#     model.add(Dropout(dropout_rate))
#     model.compile(loss='categorical_crossentropy', optimizer='Adadelta', metrics=['accuracy']) 
    
#     return model


# # Create the dropout model
# dropout_model = KerasClassifier(build_fn=create_dropout_model, epochs=250, batch_size=88)


# # Define the dropout grid search parameters
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


# # Assign the dropout grid search parameters, fit, and run the model
# dropout_param_grid = dict(dropout_rate=dropout_rate)
# dropout_grid = GridSearchCV(estimator=dropout_model, param_grid=dropout_param_grid, n_jobs=-1)
# dropout_grid_result = dropout_grid.fit(X_train_scaled, y_train_categorical)

Epoch 1/250
5243/5243 [==============================] - 0s 70us/step - loss: 0.5646 - acc: 0.6981
Epoch 2/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.4402 - acc: 0.7646
Epoch 3/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.4313 - acc: 0.7831
Epoch 4/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.4262 - acc: 0.7910
Epoch 5/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.4215 - acc: 0.8028
Epoch 6/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.4224 - acc: 0.8026
Epoch 7/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.4206 - acc: 0.8020
Epoch 8/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.4181 - acc: 0.8049
Epoch 9/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.4186 - acc: 0.8043
Epoch 10/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.4180 - acc: 0.8047
Epoch 11/

5243/5243 [==============================] - 0s 14us/step - loss: 0.3560 - acc: 0.8505
Epoch 84/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.3651 - acc: 0.8438
Epoch 85/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3557 - acc: 0.8451
Epoch 86/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3503 - acc: 0.8531
Epoch 87/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3535 - acc: 0.8514
Epoch 88/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3525 - acc: 0.8552
Epoch 89/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.3554 - acc: 0.8512
Epoch 90/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3531 - acc: 0.8539
Epoch 91/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3479 - acc: 0.8529
Epoch 92/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3460 - acc: 0.8560
Epoch 93/250


5243/5243 [==============================] - 0s 13us/step - loss: 0.3284 - acc: 0.8606
Epoch 246/250
5243/5243 [==============================] - 0s 13us/step - loss: 0.3317 - acc: 0.8617
Epoch 247/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.3344 - acc: 0.8583
Epoch 248/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.3242 - acc: 0.8652
Epoch 249/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.3345 - acc: 0.8602
Epoch 250/250
5243/5243 [==============================] - 0s 14us/step - loss: 0.3316 - acc: 0.8604


In [22]:
# # Print the score and best params for the dropout model
# print("Best: %f using %s" % (dropout_grid_result.best_score_, dropout_grid_result.best_params_))

Best: 0.623117 using {'dropout_rate': 0.0}


# Train the Model



In [ ]:
# print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")



Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

# Save the Model